<a href="https://colab.research.google.com/github/ExpertsVision/Hobby/blob/master/FaceDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Import Packages
from google.colab import drive
import numpy as np
import scipy.io
import pandas as pd
from datetime import datetime, timedelta
import os
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Dense, Activation, Dropout, Flatten, Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Activation
from keras.layers import Conv2D, AveragePooling2D
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from keras import metrics
from keras.models import model_from_json
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array, ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input
from os import listdir
from glob import glob
import math
import shutil
import re


Using TensorFlow backend.


**Downloading IMDB Dataset from Google Drive link**

We are using Google drive sharable link from https://drive.google.com/uc?id=1-7vXBlCordEUBxZanZPWYXnfPpW-lzIV&export=download
You can also download this file directly from original link i.e. https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar

In [8]:
!gdown --id 1-7vXBlCordEUBxZanZPWYXnfPpW-lzIV&export=download

Downloading...
From: https://drive.google.com/uc?id=1-7vXBlCordEUBxZanZPWYXnfPpW-lzIV
To: /content/imdb_crop.tar
7.01GB [01:59, 58.6MB/s]


In [10]:
#Unzip imdb_crop.tar
!tar -xf imdb_crop.tar

In [11]:
mat = scipy.io.loadmat('/content/imdb_crop/imdb.mat')


In [19]:
try:
  instances = mat['imdb'][0][0][0].shape[1]
  
  columns = ["dob", "photo_taken", "full_path", "gender", "name", "face_location", "face_score", "second_face_score"]
  
  df = pd.DataFrame(index = range(0,instances), columns = columns)
  
  for i in mat:
    if i == "imdb":
      current_array = mat[i][0][0]
      for j in range(len(current_array)):
        df[columns[j]] = pd.DataFrame(current_array[j][0])
except:
  pass

In [20]:
df.head()

,dob,photo_taken,full_path,gender,name,face_location,face_score,second_face_score
0,693726,1968,[01/nm0000001_rm124825600_1899-5-10_1968.jpg],1.0,[Fred Astaire],"[[1072.926, 161.838, 1214.7839999999999, 303.6...",1.459693,1.118973
1,693726,1970,[01/nm0000001_rm3343756032_1899-5-10_1970.jpg],1.0,[Fred Astaire],"[[477.184, 100.352, 622.592, 245.76]]",2.543198,1.852008
2,693726,1968,[01/nm0000001_rm577153792_1899-5-10_1968.jpg],1.0,[Fred Astaire],"[[114.96964308962852, 114.96964308962852, 451....",3.455579,2.985660
3,693726,1968,[01/nm0000001_rm946909184_1899-5-10_1968.jpg],1.0,[Fred Astaire],"[[622.8855056426588, 424.21750383700805, 844.3...",1.872117,NaN
4,693726,1968,[01/nm0000001_rm980463616_1899-5-10_1968.jpg],1.0,[Fred Astaire],"[[1013.8590023603723, 233.8820422075853, 1201....",1.158766,NaN


In [21]:
df.describe()

,dob,photo_taken,gender,face_score,second_face_score
count,460723.000000,460723.000000,452261.000000,4.607230e+05,213797.000000
mean,718987.731774,2005.461555,0.581996,-inf,2.452904
std,13253.963535,9.054475,0.493231,NaN,1.064432
min,47.000000,1961.000000,0.000000,-inf,0.730926
25%,716370.000000,2004.000000,0.000000,1.757891e+00,1.583692
50%,719935.000000,2008.000000,1.000000,2.980097e+00,2.355163
75%,723073.000000,2011.000000,1.000000,4.006376e+00,3.228071
max,734963.000000,2015.000000,1.000000,7.381689e+00,6.395435


In [15]:
df.tail()

,dob,photo_taken,full_path,gender,name,face_location,face_score,second_face_score
460718,726831,2011,[08/nm3994408_rm761245696_1989-12-29_2011.jpg],0.0,[Jane Levy],"[[453.8981431333457, 77.96623712908011, 539.79...",3.845884,NaN
460719,726831,2011,[08/nm3994408_rm784182528_1989-12-29_2011.jpg],0.0,[Jane Levy],"[[1, 1, 426, 640]]",-inf,NaN
460720,726831,2011,[08/nm3994408_rm926592512_1989-12-29_2011.jpg],0.0,[Jane Levy],"[[1, 1, 453, 640]]",-inf,NaN
460721,726831,2011,[08/nm3994408_rm943369728_1989-12-29_2011.jpg],0.0,[Jane Levy],"[[144.75225471724875, 126.76472287759263, 305....",4.450725,NaN
460722,726831,2011,[08/nm3994408_rm976924160_1989-12-29_2011.jpg],0.0,[Jane Levy],"[[457.524, 41.748, 518.016, 102.24]]",2.133503,NaN


In [22]:
#remove pictures does not include face
df = df[df['face_score'] != -np.inf]
df.describe()

,dob,photo_taken,gender,face_score,second_face_score
count,398421.000000,398421.000000,391197.000000,398421.000000,213797.000000
mean,718924.008915,2005.430873,0.572279,3.243474,2.452904
std,13177.362305,9.083973,0.494749,1.222315,1.064432
min,47.000000,1961.000000,0.000000,0.730919,0.730926
25%,716333.000000,2004.000000,0.000000,2.325559,1.583692
50%,719863.000000,2008.000000,1.000000,3.254706,2.355163
75%,722991.000000,2011.000000,1.000000,4.160408,3.228071
max,734181.000000,2015.000000,1.000000,7.381689,6.395435


In [23]:
# Only consider images in which there is no 2nd face, i.e second_face_score is nan
df = df[df['second_face_score'].isna()]
df.describe()

,dob,photo_taken,gender,face_score,second_face_score
count,184624.000000,184624.000000,181690.000000,184624.000000,0.0
mean,719075.218785,2005.815322,0.550399,3.094982,NaN
std,13659.317612,8.515250,0.497455,1.362412,NaN
min,75.000000,1961.000000,0.000000,0.730919,NaN
25%,716509.000000,2004.000000,0.000000,1.960643,NaN
50%,720002.000000,2008.000000,1.000000,3.067435,NaN
75%,723097.000000,2011.000000,1.000000,4.175454,NaN
max,734181.000000,2015.000000,1.000000,7.381689,NaN


In [24]:
# Only select pictures where face_score >3
df = df[df['face_score'] >= 3]
df.describe()

,dob,photo_taken,gender,face_score,second_face_score
count,95234.000000,95234.000000,93526.000000,95234.000000,0.0
mean,719600.493794,2006.359504,0.485961,4.213789,NaN
std,12963.520343,7.696399,0.499806,0.782275,NaN
min,75.000000,1961.000000,0.000000,3.000004,NaN
25%,717169.750000,2004.000000,0.000000,3.556245,NaN
50%,720426.000000,2008.000000,0.000000,4.137897,NaN
75%,723383.000000,2011.000000,1.000000,4.787703,NaN
max,733282.000000,2015.000000,1.000000,7.381689,NaN


**Copying filtered images in another folder**

In [26]:
# Creating folder to save preprocessed data
directory= '/content/filtered_dataset/'
try:
  os.makedirs(directory)
except OSError:
  pass

In [44]:
for i in range(df.shape[0]):
  image_path = '/content/imdb_crop/%s'%df.iloc[i,2][0]
  # Here we are only taking path of images whose names are in our fitlered df
  shutil.copy(image_path, directory)  # copying filtered images to directory

**Face Cropping using MTCNN**

In [59]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 2.8MB/s 


In [60]:
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
import cv2
from imutils import paths  
import numpy as np
import os, errno

In [62]:
detector=MTCNN()

In [65]:
def extract_face(filename, required_size=(160, 160)):
	img=cv2.imread(filename)
	pixels = np.array(img)
	# pixels = np.array(img)

	results = detector.detect_faces(pixels)
	try:
		   if results[0]['confidence']>0.60:
        		x1, y1, width, height = results[0]['box']
        	# bug fix
        		x1, y1 = abs(x1), abs(y1)
        		x2, y2 = x1 + width, y1 + height
        		face = img[y1:y2, x1:x2]
        	# image = Image.fromarray(face)
        	# face_array = asarray(image)
        		return face
	except:
			pass
			return[]

In [ ]:
folders=list(paths.list_images('/content/filtered_dataset')) #path of images folder
count=0
crop='crop'
for i,filename in enumerate(folders):
    folder = filename.split(os.path.sep)[-2].split(".")[0]
    imageName = filename.split(os.path.sep)[-1].split(".")[0]
    #print(folder)
    try:
        os.makedirs(crop+'/'+folder)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    folder_path=crop+'/'+folder						
    face=extract_face(filename)
    try:
					if(face!=[]):
							count=count+1
							pathwrite=folder_path+'/'+imageName+'crop'+str(count)+'.jpg'
							cv2.imwrite(pathwrite,face)
							#print('write',pathwrite)
    except:
					pass        

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  app.launch_new_instance()
